# CARGA DOS DADOS PARA A BRONZE
*ANÁLISE, TRATAMENTO E LIMPEZA*

In [ ]:
!pip install pyarrow
!pip install fastparquet

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob

In [2]:
# Definindo os diretórios de origem e destino
origem = 'C:\\Users\\jhoe.hashimoto\\Desktop\\v2dados_teste\\transient'
destino = 'C:\\Users\\jhoe.hashimoto\\Desktop\\v2dados_teste\\bronze'

In [3]:
# Lista todos os arquivos no diretório
arquivos = os.listdir(origem)

# Filtra apenas os arquivos .xlsx ou excel
arquivos_xlsx = [arquivo for arquivo in arquivos if arquivo.endswith('.xlsx')]

# Lista para armazenar os DataFrames
dataframes = []

# Lê cada arquivo .xlsx e armazena na lista
for arquivo in arquivos_xlsx:
    caminho_completo = os.path.join(origem, arquivo)
    df = pd.read_excel(caminho_completo)
    dataframes.append(df)

    
d1, d2, d3, d4= dataframes


*IMPOSSIBILIDADE DE SALVAR OS ARQUIVOS DIRETAMENTE DE PARQUET, NECESSIDADE DE CONVERTER PARA OS TIPOS CORRETOS*

**POSSUI A PRIMEIRA LINHA SEM REGISTROS E CÉLULAS MESCLADAS**

*iremos APAGAR o Cabeçalho, inferindo o cabeçalho para próxima linha*

In [4]:
for i, df in enumerate(dataframes):
    # Verificar se o cabeçalho contém a palavra 'Unnamed'
    if 'Unnamed' in df.columns[0]:
        # Renomear colunas com os valores da primeira linha
        df.columns = df.iloc[0]
        # Remover a primeira linha
        df.drop(df.index[0], axis=0, inplace=True)
          # Atualizar o dataframe na lista
        dataframes[i] = df
      

In [5]:
d4.head()

,order_id,product_name,quantity,unit_price,cliente_id,order_date
1,1,Produto A,3,25,74,2024-03-01 00:00:00
2,2,Produto B,2,30,36,2024-03-02 00:00:00
3,3,Produto C,1,20,12,2024-03-03 00:00:00
4,4,Produto A,4,25,92,2024-03-04 00:00:00
5,5,Produto D,2,40,55,2024-03-05 00:00:00


*TESTANDO O RESULTADO DA OPERAÇÃO NO CABEÇALHO*

In [265]:
#d4.to_excel('C:\\Users\\jhoe.hashimoto\\Desktop\\v2dados_teste\\bronze\\teste2.xlsx', index=False)

In [6]:
d2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_cliente     220 non-null    int64 
 1   nome           220 non-null    object
 2   sobrenome      220 non-null    object
 3   email          220 non-null    object
 4   telefone       220 non-null    object
 5   endereço       220 non-null    object
 6   numero         220 non-null    int64 
 7   Data_Cadastro  220 non-null    object
dtypes: int64(2), object(6)
memory usage: 13.9+ KB


***ANÁLISE DAS COLUNAS DE DATA***
#presença de strings, datas com formato incorreto#

In [7]:
d2.tail(n=6)

,id_cliente,nome,sobrenome,email,telefone,endereço,numero,Data_Cadastro
214,215,Marcelo,Silva,marcelo.silva@email.com,(11) 2345-6789,Avenida dos Narcisos,654,18/10/20233
215,216,Aline,Santos,aline.santos@email.com,(11) 3456-7890,Travessa das Tulipas,321,2021-02-05 00:00:00
216,217,Lucas,Lima,lucas.lima@email.com,(11) 4567-8901,Rua dos Girassóis,987,2023-08-03 00:00:00
217,218,Isabella,Campos,isabella.campos@email.com,(11) 5678-9012,Alameda dos Crisântemos,654,2023-09-09 00:00:00
218,219,Mateus,Almeida,mateus.almeida@email.com,(11) 6789-0123,Quadra 11,321,44348
219,220,Larissa,Gonçalves,larissa.goncalves@email.com,(11) 7890-1234,Praça dos Jardins,987,2021-03-18 00:00:00


In [8]:
#verificando tipo
tipo_coluna = d2['Data_Cadastro'].dtype
tipo_coluna

dtype('O')

**CONVERSÃO DO TIPO PARA TRATAMENTO**

In [9]:
#tipando para string, para realizar alterações
if 'Data_Cadastro' in d2.columns:
    d2['Data_Cadastro'] = d2['Data_Cadastro'].astype(str)
d2['Data_Cadastro'].tail(n=6)    

214            18/10/20233
215    2021-02-05 00:00:00
216    2023-08-03 00:00:00
217    2023-09-09 00:00:00
218                  44348
219    2021-03-18 00:00:00
Name: Data_Cadastro, dtype: object

In [10]:
#aplicando formatação de string para uma lista de 18 caracteres, caso haja mais iremos apagar o último índice
if 'Data_Cadastro' in d2.columns:
    d2['Data_Cadastro'] = d2['Data_Cadastro'].apply(lambda x: x[:-1] if len(x) < 18 else x)

In [11]:
d2.tail(n=6)

,id_cliente,nome,sobrenome,email,telefone,endereço,numero,Data_Cadastro
214,215,Marcelo,Silva,marcelo.silva@email.com,(11) 2345-6789,Avenida dos Narcisos,654,18/10/2023
215,216,Aline,Santos,aline.santos@email.com,(11) 3456-7890,Travessa das Tulipas,321,2021-02-05 00:00:00
216,217,Lucas,Lima,lucas.lima@email.com,(11) 4567-8901,Rua dos Girassóis,987,2023-08-03 00:00:00
217,218,Isabella,Campos,isabella.campos@email.com,(11) 5678-9012,Alameda dos Crisântemos,654,2023-09-09 00:00:00
218,219,Mateus,Almeida,mateus.almeida@email.com,(11) 6789-0123,Quadra 11,321,4434
219,220,Larissa,Gonçalves,larissa.goncalves@email.com,(11) 7890-1234,Praça dos Jardins,987,2021-03-18 00:00:00


***APLICANDO A TIPAGEM FINAL***

In [ ]:
#Aplicando a tipagem de data para o campo 'Data_Cadastro'
for df in dataframes:
    if 'Data_Cadastro' in df:
        # Convertendo para datetime e formatando para 'YYYY-MM-DD'
        df['Data_Cadastro'] = pd.to_datetime(df['Data_Cadastro'], errors='coerce').dt.date   #corece passa NAN, para casos onde não há correspondência
        
        # Preenchendo valores NaN com '31/12/2099' ---> outlier definido
        df['Data_Cadastro'].fillna(pd.to_datetime('2099-12-31').date(), inplace=True)


In [ ]:
#Aplicando a tipagem de data para o campo 'order_date'
for df in dataframes:
    if 'order_date' in df:
        df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce').dt.date
         # Preenchendo valores NaN com '31/12/2099' ---> outlier definido
        df['order_date'].fillna(value=pd.to_datetime('31/12/2099').date(), inplace=True)

In [ ]:
#Formatando o campo 'unit_price', no qual continha strings
for df in dataframes:
    if 'unit_price' in df:
        df['unit_price'] = pd.to_numeric(df['unit_price'], errors='coerce')
        # *APLICANDO PREÇO = 0 PARA OS VALORES NULOS*
        df['unit_price'].fillna(0, inplace=True)

In [17]:
d3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_id      100 non-null    int64 
 1   product_name  100 non-null    object
 2   quantity      100 non-null    int64 
 3   unit_price    100 non-null    int64 
 4   cliente_id    100 non-null    int64 
 5   order_date    100 non-null    object
dtypes: int64(4), object(2)
memory usage: 4.8+ KB


In [18]:
d2.tail(n=10)

,id_cliente,nome,sobrenome,email,telefone,endereço,numero,Data_Cadastro
210,211,Roberto,Campos,roberto.campos@email.com,(11) 8901-2345,Rua das Margaridas,987,2020-03-25
211,212,Ana,Cavalcanti,ana.cavalcanti@email.com,(11) 9012-3456,Avenida dos Cravos,654,2021-06-25
212,213,Renato,Fernandes,renato.fernandes@email.com,(11) 0123-4567,Alameda dos Lírios,321,2021-05-25
213,214,Priscila,Ferreira,priscila.ferreira@email.com,(11) 1234-5678,Rua das Violetas,987,2022-12-13
214,215,Marcelo,Silva,marcelo.silva@email.com,(11) 2345-6789,Avenida dos Narcisos,654,2099-12-31
215,216,Aline,Santos,aline.santos@email.com,(11) 3456-7890,Travessa das Tulipas,321,2021-02-05
216,217,Lucas,Lima,lucas.lima@email.com,(11) 4567-8901,Rua dos Girassóis,987,2023-08-03
217,218,Isabella,Campos,isabella.campos@email.com,(11) 5678-9012,Alameda dos Crisântemos,654,2023-09-09
218,219,Mateus,Almeida,mateus.almeida@email.com,(11) 6789-0123,Quadra 11,321,2099-12-31
219,220,Larissa,Gonçalves,larissa.goncalves@email.com,(11) 7890-1234,Praça dos Jardins,987,2021-03-18


In [20]:
d4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 1 to 166
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      166 non-null    object 
 1   product_name  166 non-null    object 
 2   quantity      166 non-null    object 
 3   unit_price    166 non-null    float64
 4   cliente_id    166 non-null    object 
 5   order_date    166 non-null    object 
dtypes: float64(1), object(5)
memory usage: 7.9+ KB


In [320]:
# d2.to_excel("C:\\Users\\jhoe.hashimoto\\Desktop\\v2dados_teste\\bronze\\tested2.xlsx", index=False)
# d4.to_excel("C:\\Users\\jhoe.hashimoto\\Desktop\\v2dados_teste\\bronze\\tested4.xlsx", index=False)

**PADRONIZAÇÃO NOME COLUNAS**

In [21]:
#RENOMEIO E PADRONIZAÇÃO DAS COLUNAS ID_CLIENTE
for df in dataframes:
    if 'cliente_id' in df.columns:
        df.rename(columns={'cliente_id': 'id_cliente'}, inplace=True)
        

***COMPARAÇÃO DO TIPO DE DADO EM CADA VERSÃO DE DATAFRAME CORRESPONDENTE***

In [22]:
customer1, customer2, sales1, sales2= dataframes


dtypes1 = customer1.dtypes
dtypes2 = customer2.dtypes

# Criando um DataFrame de comparação para todas as colunas
comparison_customer = pd.DataFrame({
    'customer1': dtypes1,
    'customer2': dtypes2
})
comparison_customer['Mesmo tipo'] = comparison_customer['customer1'] == comparison_customer['customer2']
comparison_customer['ColunaNova?'] = comparison_customer.apply(
    lambda row: 'Nova Coluna' if pd.isnull(row['customer2']) else ('NovaColuna' if pd.isnull(row['customer1']) else 'Presente'), axis=1
)
comparison_customer

,customer1,customer2,Mesmo tipo,ColunaNova?
Data_Cadastro,NaN,object,False,NovaColuna
email,object,object,True,Presente
endereço,object,object,True,Presente
id_cliente,int64,int64,True,Presente
nome,object,object,True,Presente
numero,int64,int64,True,Presente
sobrenome,object,object,True,Presente
telefone,object,object,True,Presente


***CONVERSÃO DA COLUNA 'Data_Cadastro' PARA O TIPO DATE***

In [23]:
customer2['Data_Cadastro'] = pd.to_datetime(customer2['Data_Cadastro'], format='%Y-%m-%d')

In [24]:
dtypes3 = sales1.dtypes
dtypes4 = sales2.dtypes

# Criando um DataFrame de comparação para todas as colunas
comparison_customer2 = pd.DataFrame({
    'sales1': dtypes3,
    'sales2': dtypes4
})
comparison_customer2['Mesmo tipo'] = comparison_customer2['sales1'] == comparison_customer2['sales2']
comparison_customer2['ColunaNova?'] = comparison_customer2.apply(
    lambda row: 'Nova Coluna' if pd.isnull(row['sales2']) else ('NovaColuna' if pd.isnull(row['sales1']) else 'Presente'), axis=1
)
comparison_customer2

,sales1,sales2,Mesmo tipo,ColunaNova?
order_id,int64,object,False,Presente
product_name,object,object,True,Presente
quantity,int64,object,False,Presente
unit_price,int64,float64,False,Presente
id_cliente,int64,object,False,Presente
order_date,object,object,True,Presente


***CONVERSÃO DOS TIPOS NUMÉRICOS PARA A TIPAGEM CORRETA***

In [25]:
sales1['unit_price'] = sales1['unit_price'].astype('float64')
sales1['order_date'] = pd.to_datetime(sales1['order_date'], format='%Y-%m-%d')


sales2['order_id'] = sales2['order_id'].astype('int64')
sales2['quantity'] = sales2['quantity'].astype('int64')
sales2['id_cliente'] = sales2['id_cliente'].astype('int64')
sales2['order_date'] = pd.to_datetime(sales2['order_date'], format='%Y-%m-%d')


***PRINTANDO RESULTADOS DA OPERAÇÃO***

In [26]:
comparison_customer

,customer1,customer2,Mesmo tipo,ColunaNova?
Data_Cadastro,NaN,object,False,NovaColuna
email,object,object,True,Presente
endereço,object,object,True,Presente
id_cliente,int64,int64,True,Presente
nome,object,object,True,Presente
numero,int64,int64,True,Presente
sobrenome,object,object,True,Presente
telefone,object,object,True,Presente


In [27]:
comparison_customer2

,sales1,sales2,Mesmo tipo,ColunaNova?
order_id,int64,object,False,Presente
product_name,object,object,True,Presente
quantity,int64,object,False,Presente
unit_price,int64,float64,False,Presente
id_cliente,int64,object,False,Presente
order_date,object,object,True,Presente


In [28]:
customer_final1, customer_final2, sales_final1, sales_final2 = dataframes

***SALVANDO OS DADOS EM PARQUET***

In [ ]:
path_excel = 'C:\\Users\\jhoe.hashimoto\\Desktop\\v2dados_teste\\bronze\\xlsx'
path_parquet = 'C:\\Users\\jhoe.hashimoto\\Desktop\\v2dados_teste\\bronze\\parquet'


os.makedirs(path_excel, exist_ok=True)

nomes_arquivos = ['customer_final1', 'customer_final2', 'sales_final1', 'sales_final2']

# Lista de dataframes
dataframes = [customer_final1, customer_final2, sales_final1, sales_final2]

# Salvando os dataframes
for nome, df in zip(nomes_arquivos, dataframes):
    nome_arquivo_excel = os.path.join(path_excel, f'{nome}.xlsx')
    nome_arquivo_parquet = os.path.join(path_parquet, f'{nome}.parquet')
    
    # Salvando em Excel
    df.to_excel(nome_arquivo_excel, index=False)
    
    # Salvando em Parquet
    df.to_parquet(nome_arquivo_parquet, index=False)
    
    print(f'{nome_arquivo_excel} e {nome_arquivo_parquet} criados com sucesso.')

*CRIANDO BACKUP DA PASTA DO PROJETO*

In [1]:
import os
import shutil
from datetime import datetime

source_folder = 'C:\\Users\\jhoe.hashimoto\\Desktop\\v2dados_teste'

backup_folder = f'C:\\Users\\jhoe.hashimoto\\Desktop\\backup_v2dados_teste_{datetime.now().strftime("%Y%m%d_%H%M%S")}'

def create_backup(source, backup):
    try:
        # Copiando a pasta
        shutil.copytree(source, backup)
        print(f'Backup criado com sucesso: {backup}')
    except Exception as e:
        print(f'Erro ao criar backup: {e}')

# Criando backup
create_backup(source_folder, backup_folder)

Backup criado com sucesso: C:\Users\jhoe.hashimoto\Desktop\backup_v2dados_teste_20240620_092811
